In [ ]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

## Enrique's code starts here

In [ ]:
# read in the data
employment_data = pd.read_csv("Resources/Average annual hours actually worked.csv")
happiness_data = pd.read_csv("Resources/World Happiness Report 2018.csv")

In [ ]:
# clean up employment_data

# get only relevant rows
employment_data = employment_data[["Country", "Employment status", "Time", "Value"]]

# get only total employment
total_employment = employment_data.loc[employment_data["Employment status"] == "Total employment"]

# get only 2018 data
total_employment_2018 = total_employment.loc[total_employment["Time"] == 2018]

# sort by country
total_employment_2018 = total_employment_2018.sort_values(by="Country")

# rename columns and get a final dataframe
total_employment_2018 = total_employment_2018[["Country", "Value"]]
total_employment_2018 = total_employment_2018.rename(columns={"Value": "Avg Hours Worked per Worker"})

In [ ]:
# clean up happiness_data

# rename country column for merging
happiness_data = happiness_data.rename(columns={"Country or region": "Country"})

In [ ]:
# merge happiness data into the employment data
data = pd.merge(total_employment_2018, happiness_data, how="left", on="Country")

# clean up merged data
data = data.dropna()

In [ ]:
# Plot Average Number of Hours Worked per Worker by Country

# countries list and tick locations
countries = data["Country"].tolist()
tick_locations = np.arange(len(countries))

# dataframe for plot
hours_worked = data[["Country", "Avg Hours Worked per Worker"]]

# plot the data
hours_worked.plot(kind="bar", legend=False, width=.8)

# assign xticks
plt.xticks(tick_locations, countries, rotation=90)

# title and labels
plt.title(f"Average Number of Hours Worked\n"
          f"per Worker by Country (2018)")
plt.xlabel("Country")
plt.ylabel("Number of Hours")

print(len(countries))
# display plot
plt.tight_layout()
plt.show()

In [ ]:
# Regression Analysis for Question #1
# Hypothesis: Countries with lower working hours will tend to have a higher happiness score.

# define independent and dependent variables
independent_variable = "Avg Hours Worked per Worker"
dependent_variable = "Score"

# define title and labels
xlabel = "Number of Hours Worked"
ylabel = "Happiness Score"
title = f"{ylabel} vs. {xlabel}"

# create dataframe
hours_and_happiness = data[["Country", independent_variable, dependent_variable]]

# define x and y values
x_values = hours_and_happiness[independent_variable]
y_values = hours_and_happiness[dependent_variable]

# run regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)

# calculate regression values
regress_values = x_values * slope + intercept

# create line equation
line_eq = f"y = {round(slope, 4)}x + {round(intercept, 2)}"

# plot the data
plt.scatter(x_values, y_values)

# plot the regression line
plt.plot(x_values, regress_values, "r-")

# add equation
plt.annotate(line_eq, (1780, 5.8), fontsize=15, color="red")

# add title and labels to plot
plt.title(title)
plt.xlabel(xlabel)
plt.ylabel(ylabel)

# print summary data
print(f"The r-squared value is: {round(rvalue**2, 2)}")
print(f"The p-value is: {round(pvalue, 2)}")

# display the plot
plt.show()

### Enrique's code ends here

## Jake's code starts here

In [ ]:
#read in ILO data

ilo_data = pd.read_csv("Resources/ILOSTAT.csv")

#merge data frames
#You need to run Enrique's code before this cell to create happiness_data and rename the country column
ilo_happiness_df = pd.merge(ilo_data, happiness_data, on="Country")




ilo_happiness_df.head()

In [ ]:
#separate countries into income groups
low_income = ilo_happiness_df[ilo_happiness_df["Group"] == "Low income countries"]
low_mid_income = ilo_happiness_df[ilo_happiness_df["Group"] == "Lower-middle income countries"]
up_mid_income = ilo_happiness_df[ilo_happiness_df["Group"] == "Upper-middle income countries"]
high_income = ilo_happiness_df[ilo_happiness_df["Group"] == "High income countries"]

#calculate mean happiness score for each income group
low_income_happiness = low_income["Score"].mean()
low_mid_income_happiness = low_mid_income["Score"].mean()
up_mid_income_happiness = up_mid_income["Score"].mean()
high_income_happiness = high_income["Score"].mean()

#create dataframe with income groups and their mean happiness score
happiness_groups = pd.DataFrame({"Income Group": ["Low Income", "Lower-Middle Income", "Upper-Middle Income", "High Income"],
                                "Average Happiness Score": [low_income_happiness, low_mid_income_happiness, up_mid_income_happiness, high_income_happiness]})

happiness_groups

In [ ]:
plt.bar(happiness_groups["Income Group"], happiness_groups["Average Happiness Score"], color='r', alpha=1, align="center")
plt.xticks(rotation=45)
plt.title(f"Average Happiness Score by Income Group")
plt.xlabel("Income Group")
plt.ylabel("Average Happiness Score")
plt.show()

In [ ]:
#happiness score vs poverty rate

poverty_df = pd.DataFrame({"Country": ilo_happiness_df["Country"],
                          "Happiness Score": ilo_happiness_df["Score"]})

for row in poverty_df:
    poverty_df["Poverty Rate"] = ilo_happiness_df["Extremely poor"] + ilo_happiness_df["Moderately poor"]
    
poverty_df = poverty_df.dropna()

poverty_df

In [ ]:
# define x and y values
x_values = poverty_df["Poverty Rate"]
y_values = poverty_df["Happiness Score"]

# run regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)

# calculate regression values
regress_values = x_values * slope + intercept

# create line equation
line_eq = f"y = {round(slope, 4)}x + {round(intercept, 2)}"

# plot the data
plt.scatter(x_values, y_values)

# plot the regression line
plt.plot(x_values, regress_values, "r-")

# add equation
plt.annotate(line_eq, (39, 5.5), fontsize=15, color="red")

# add title and labels to plot
plt.title("Happiness Score vs. Poverty Rate")
plt.xlabel("Poverty Rate")
plt.ylabel("Happiness Score")

# print summary data
print(f"The r-squared value is: {round(rvalue**2, 2)}")
print(f"The p-value is: {round(pvalue, 2)}")

# display the plot
plt.show()

### Jake's code ends here

## Yee's code starts here

### Yee's code ends here